In [0]:
dbutils.fs.mount(source='wasbs://inputdatasets@testdeltalake.blob.core.windows.net',mount_point='/mnt/retaildb',
extra_configs={'fs.azure.account.key.testdeltalake.blob.core.windows.net':'qeTzhEEtoKjdZv/H0qQ+CFRci6mOKjPAekVY5mxFDRB5/78rcHZK3if+JbVGs0dHqmV6lLmNiRlb+AStatC5BA=='})

True

In [0]:
df=spark.read.format("csv").option("header","true").load("dbfs:/mnt/retaildb/orders.csv")

In [0]:
df.printSchema()

root
 |-- 1: string (nullable = true)
 |-- 2013-07-25 00:00:00.0: string (nullable = true)
 |-- 11599: string (nullable = true)
 |-- CLOSED: string (nullable = true)



In [0]:
order_schema='order_id string,order_date string,customer_id string,order_status string'

In [0]:
df=spark.read.format("csv").schema(order_schema).option("header","true").load("dbfs:/mnt/retaildb/orders.csv")

In [0]:
df.show(5)

+--------+--------------------+-----------+---------------+
|order_id|          order_date|customer_id|   order_status|
+--------+--------------------+-----------+---------------+
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|      12111|       COMPLETE|
|       4|2013-07-25 00:00:...|       8827|         CLOSED|
|       5|2013-07-25 00:00:...|      11318|       COMPLETE|
|       6|2013-07-25 00:00:...|       7130|       COMPLETE|
+--------+--------------------+-----------+---------------+
only showing top 5 rows



In [0]:
df.write.mode("overwrite").\
    partitionBy("order_status").format("parquet").save("/mnt/retaildb/parquet/orders.parquet")

In [0]:
df.write.mode("overwrite").\
    partitionBy("order_status").format("delta").save("/mnt/retaildb/delta/orders.delta")

In [0]:
%sql
create database if not exists retaildb

In [0]:
%sql
create table retaildb.orderparquet using parquet location "/mnt/retaildb/parquet/orders.parquet"

In [0]:
%sql
select * from retaildb.orderparquet limit 10

order_id,order_date,customer_id,order_status


## it will not show the results it was unable to read the files in folder we need to pass /*

In [0]:
%sql
drop table retaildb.orderparquet

In [0]:
%sql
create table retaildb.orderparquet using parquet location "/mnt/retaildb/parquet/orders.parquet/*"

In [0]:
%sql
select * from retaildb.orderparquet limit 10

order_id,order_date,customer_id
68883,2014-07-23 00:00:00.0,5533
68880,2014-07-13 00:00:00.0,1117
68879,2014-07-09 00:00:00.0,778
68878,2014-07-08 00:00:00.0,6753
68876,2014-07-06 00:00:00.0,4124
68874,2014-07-03 00:00:00.0,1601
68872,2014-06-29 00:00:00.0,3354
68870,2014-06-26 00:00:00.0,3343
68859,2014-06-11 00:00:00.0,1428
68858,2014-06-06 00:00:00.0,10744


### create table using delta format

In [0]:
%sql
create table retaildb.orderdelta using delta location "/mnt/retaildb/delta/orders.delta"

In [0]:
%sql
select * from retaildb.orderdelta limit 10

order_id,order_date,customer_id,order_status
68865,2014-06-19 00:00:00.0,4567,SUSPECTED_FRAUD
68782,2014-01-10 00:00:00.0,8509,SUSPECTED_FRAUD
68718,2013-09-14 00:00:00.0,6710,SUSPECTED_FRAUD
68686,2014-07-23 00:00:00.0,2591,SUSPECTED_FRAUD
68606,2014-06-28 00:00:00.0,2253,SUSPECTED_FRAUD
68555,2014-06-14 00:00:00.0,4454,SUSPECTED_FRAUD
68534,2014-06-08 00:00:00.0,5168,SUSPECTED_FRAUD
68522,2014-06-05 00:00:00.0,880,SUSPECTED_FRAUD
68506,2014-05-29 00:00:00.0,10527,SUSPECTED_FRAUD
68416,2014-05-04 00:00:00.0,7102,SUSPECTED_FRAUD


In [0]:
%sql
drop table retaildb.orderdelta

In [0]:
%sql
describe table extended retaildb.orderdelta

col_name,data_type,comment
order_id,string,null
order_date,string,null
customer_id,string,null
order_status,string,null
# Partition Information,,
# col_name,data_type,comment
order_status,string,null
,,
# Delta Statistics Columns,,
Column Names,"order_id, order_date, customer_id",


In [0]:
df.write.mode("overwrite").partitionBy("order_status").format("delta").option("path","/mnt/retaildb/deltalatest/orders.delta").saveAsTable("retaildb.orderdeltatable")

In [0]:
%sql
select * from retaildb.orderdeltatable limit 10

order_id,order_date,customer_id,order_status
68867,2014-06-23 00:00:00.0,869,CANCELED
68816,2014-03-26 00:00:00.0,8769,CANCELED
68792,2014-01-28 00:00:00.0,9809,CANCELED
68767,2013-12-11 00:00:00.0,6671,CANCELED
68754,2013-11-16 00:00:00.0,2424,CANCELED
68743,2013-11-01 00:00:00.0,2751,CANCELED
68571,2014-06-19 00:00:00.0,2796,CANCELED
68557,2014-06-14 00:00:00.0,10109,CANCELED
68528,2014-06-06 00:00:00.0,10183,CANCELED
68516,2014-06-03 00:00:00.0,5372,CANCELED


In [0]:
df.write.mode("overwrite").partitionBy("order_status").format("delta").option("path","/mnt/retaildb/deltalatest/orders.delta").saveAsTable("retaildb.orderdeltatable")

In [0]:
%sql
insert into retaildb.orderdeltatable values('1111111','2014-06-03 00:00:00.0','7093967527','CLOSED')

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
select * from retaildb.orderdeltatable where customer_id='7093967527'

order_id,order_date,customer_id,order_status
1111111,2014-06-03 00:00:00.0,7093967527,CLOSED


In [0]:
%sql
describe history retaildb.orderdeltatable

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2024-03-16T12:22:33Z,4110497945053574,jeswanthnaidu27@outlook.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2636702334976547),0316-095601-y7ipfe71,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1053)",null,Databricks-Runtime/13.3.x-photon-scala2.12
1,2024-03-16T12:13:41Z,4110497945053574,jeswanthnaidu27@outlook.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [""order_status""], description -> null, isManaged -> false, properties -> {}, statsOnLoad -> false)",null,List(2636702334976547),0316-095601-y7ipfe71,0,WriteSerializable,false,"Map(numFiles -> 9, numOutputRows -> 68882, numOutputBytes -> 675647)",null,Databricks-Runtime/13.3.x-photon-scala2.12
0,2024-03-16T12:08:39Z,4110497945053574,jeswanthnaidu27@outlook.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [""order_status""], description -> null, isManaged -> false, properties -> {}, statsOnLoad -> false)",null,List(2636702334976547),0316-095601-y7ipfe71,null,WriteSerializable,false,"Map(numFiles -> 9, numOutputRows -> 68882, numOutputBytes -> 675647)",null,Databricks-Runtime/13.3.x-photon-scala2.12


In [0]:
df_append=spark.read.format("csv").schema(order_schema).load("dbfs:/mnt/retaildb/ordersAppend.csv")

In [0]:
df.write.mode("append").partitionBy("order_status").format("delta").option("path","dbfs:/mnt/retaildb/ordersAppend.csv").saveAsTable("retaildb.orderdeltatable")

In [0]:
%sql
describe history retaildb.orderdeltatable

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
3,2024-03-16T12:34:30Z,4110497945053574,jeswanthnaidu27@outlook.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2636702334976547),0316-095601-y7ipfe71,2,WriteSerializable,true,"Map(numFiles -> 9, numOutputRows -> 68882, numOutputBytes -> 675647)",null,Databricks-Runtime/13.3.x-photon-scala2.12
2,2024-03-16T12:22:33Z,4110497945053574,jeswanthnaidu27@outlook.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2636702334976547),0316-095601-y7ipfe71,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1053)",null,Databricks-Runtime/13.3.x-photon-scala2.12
1,2024-03-16T12:13:41Z,4110497945053574,jeswanthnaidu27@outlook.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [""order_status""], description -> null, isManaged -> false, properties -> {}, statsOnLoad -> false)",null,List(2636702334976547),0316-095601-y7ipfe71,0,WriteSerializable,false,"Map(numFiles -> 9, numOutputRows -> 68882, numOutputBytes -> 675647)",null,Databricks-Runtime/13.3.x-photon-scala2.12
0,2024-03-16T12:08:39Z,4110497945053574,jeswanthnaidu27@outlook.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [""order_status""], description -> null, isManaged -> false, properties -> {}, statsOnLoad -> false)",null,List(2636702334976547),0316-095601-y7ipfe71,null,WriteSerializable,false,"Map(numFiles -> 9, numOutputRows -> 68882, numOutputBytes -> 675647)",null,Databricks-Runtime/13.3.x-photon-scala2.12


#### insrting data using copy command

In [0]:
%sql
copy into retaildb.orderdeltatable from "dbfs:/mnt/retaildb/order1.csv" format_options('header','True')

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: org.apache.spark.sql.catalyst.parser.ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'format_options'.(line 1, pos 72)

== SQL ==
copy into retaildb.orderdeltatable from "dbfs:/mnt/retaildb/order1.csv" format_options('header','True')
------------------------------------------------------------------------^^^

	at org.apache.spark.sql.catalyst.parser.ParseException.withCommand(parsers.scala:259)
	at org.apache.spark.sql.catalyst.parser.AbstractParser.parse(parsers.scala:98)
	at org.apache.spark.sql.execution.SparkSqlParser.parse(SparkSqlParser.scala:111)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parsePlan(AbstractSqlParser.scala:87)
	at com.databricks.sql.parser.DatabricksSqlParser.$anonfun$parsePlan$1(DatabricksSqlParser.scala:77)
	at com.databricks.sql.parser.DatabricksSqlParser.parse(DatabricksSqlParser.scala:98)
	at com.databricks.sql.parser.DatabricksSqlParser.parsePlan(DatabricksSqlParser.scala:74)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$6(SparkSession.scala:919)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:400)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$5(SparkSession.scala:918)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:1148)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:917)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$9(SparkSession.scala:951)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:1148)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:951)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:984)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:695)
	at com.databricks.backend.daemon.driver.DriverLocal$DbClassicStrategy.executeSQLQuery(DriverLocal.scala:237)
	at com.databricks.backend.daemon.driver.DriverLocal.executeSQLSubCommand(DriverLocal.scala:319)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$executeSql$1(DriverLocal.scala:338)
	at scala.collection.immutable.List.map(List.scala:293)
	at com.databricks.backend.daemon.driver.DriverLocal.executeSql(DriverLocal.scala:333)
	at com.databricks.backend.daemon.driver.JupyterDriverLocal.repl(JupyterDriverLocal.scala:709)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$33(DriverLocal.scala:997)
	at com.databricks.unity.EmptyHandle$.runWith(UCSHandle.scala:125)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$22(DriverLocal.scala:980)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:426)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:196)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:418)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:69)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:470)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:455)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:69)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:935)
	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:798)
	at scala.util.Try$.apply(Try.scala:213)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:790)
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommandAndGetError(DriverWrapper.scala:643)
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:744)
	at com.databricks.backend.dae

In [0]:
df_new=spark.read.format("csv").option("header","true").load("dbfs:/mnt/retaildb/order_new.csv")

In [0]:
df_new.show()

+--------+--------------------+-----------+---------------+------------+
|order_id|          order_date|customer_id|   order_status|order_amount|
+--------+--------------------+-----------+---------------+------------+
| 1000009|2013-07-25 00:00:...|      11599|         CLOSED|          10|
| 2000009|2013-07-25 00:00:...|        256|PENDING_PAYMENT|          50|
+--------+--------------------+-----------+---------------+------------+



In [0]:
df_new.write.mode("append").partitionBy("order_status").format("delta").save("/mnt/retaildb/deltalatest/orders.delta")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2628809400550742>, line 1
----> 1 df_new.write.mode("append").partitionBy("order_status").format("delta").save("/mnt/retaildb/deltalatest/orders.delta")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1463, in DataFrameWriter.save(self, path, format, mode, partitionBy, **options)
   1461     self._jwrite.save()
   1462 else:
-> 1463     self._jwrite.save(path)

File /databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py:1322, in Ja

In [0]:
df_new.write.mode("append").partitionBy("order_status").format("delta").option('mergeSchema','true').save("/mnt/retaildb/deltalatest/orders.delta")

In [0]:
%sql
select * from retaildb.orderdeltatable where order_amount=10 limit 10

order_id,order_date,customer_id,order_status,order_amount
1000009,2013-07-25 00:00:00.0,11599,CLOSED,10


In [0]:
%sql
update retaildb.orderdeltatable set order_amount=900 where order_amount=10

num_affected_rows
1


In [0]:
%sql
delete from retaildb.orderdeltatable where order_amount=900

num_affected_rows
1


In [0]:
%sql
select customer_id,count(*) as total from retaildb.orderdeltatable group by customer_id having total>1

customer_id,total
10351,33
1512,24
7711,24
11332,27
12394,24
829,15
9009,24
2088,15
3210,9
296,27


In [0]:
%sql
delete from retaildb.orderdeltatable where customer_id=10351

num_affected_rows
33


%% History using time travel

In [0]:
%sql
describe history retaildb.orderdeltatable

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
8,2024-03-16T13:17:05Z,4110497945053574,jeswanthnaidu27@outlook.com,DELETE,"Map(predicate -> [""(cast(customer_id#11913 as int) = 10351)""])",null,List(2636702334976547),0316-095601-y7ipfe71,7,WriteSerializable,false,"Map(numRemovedFiles -> 15, numRemovedBytes -> 1617213, numCopiedRows -> 170643, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2015, numDeletedRows -> 33, scanTimeMs -> 682, numAddedFiles -> 7, numAddedBytes -> 1181355, rewriteTimeMs -> 1332)",null,Databricks-Runtime/13.3.x-photon-scala2.12
7,2024-03-16T13:13:44Z,4110497945053574,jeswanthnaidu27@outlook.com,DELETE,"Map(predicate -> [""(cast(order_amount#11186 as int) = 900)""])",null,List(2636702334976547),0316-095601-y7ipfe71,6,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1247, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 978, numDeletedRows -> 1, scanTimeMs -> 629, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 349)",null,Databricks-Runtime/13.3.x-photon-scala2.12
6,2024-03-16T13:12:20Z,4110497945053574,jeswanthnaidu27@outlook.com,UPDATE,"Map(predicate -> [""(cast(order_amount#10692 as int) = 10)""])",null,List(2636702334976547),0316-095601-y7ipfe71,5,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1242, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 1834, scanTimeMs -> 1023, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 1247, rewriteTimeMs -> 786)",null,Databricks-Runtime/13.3.x-photon-scala2.12
5,2024-03-16T13:00:23Z,4110497945053574,jeswanthnaidu27@outlook.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [""order_status""])",null,List(2636702334976547),0316-095601-y7ipfe71,4,WriteSerializable,true,"Map(numFiles -> 2, numOutputRows -> 2, numOutputBytes -> 2474)",null,Databricks-Runtime/13.3.x-photon-scala2.12
4,2024-03-16T12:58:54Z,4110497945053574,jeswanthnaidu27@outlook.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [""order_status""])",null,List(2636702334976547),0316-095601-y7ipfe71,3,WriteSerializable,true,"Map(numFiles -> 9, numOutputRows -> 68882, numOutputBytes -> 675647)",null,Databricks-Runtime/13.3.x-photon-scala2.12
3,2024-03-16T12:34:30Z,4110497945053574,jeswanthnaidu27@outlook.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2636702334976547),0316-095601-y7ipfe71,2,WriteSerializable,true,"Map(numFiles -> 9, numOutputRows -> 68882, numOutputBytes -> 675647)",null,Databricks-Runtime/13.3.x-photon-scala2.12
2,2024-03-16T12:22:33Z,4110497945053574,jeswanthnaidu27@outlook.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(2636702334976547),0316-095601-y7ipfe71,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1053)",null,Databricks-Runtime/13.3.x-photon-scala2.12
1,2024-03-16T12:13:41Z,4110497945053574,jeswanthnaidu27@outlook.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [""order_status""], description -> null, isManaged -> false, properties -> {}, statsOnLoad -> false)",null,List(2636702334976547),0316-095601-y7ipfe71,0,WriteSerializable,false,"Map(numFiles -> 9, numOutputRows -> 68882, numOutputBytes -> 675647)",null,Databricks-Runtime/13.3.x-photon-scala2.12
0,2024-03-16T12:08:39Z,4110497945053574,jeswanthnaidu27@outlook.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [""order_status""], description -> null, isManaged -> false, properties -> {}, statsOnLoad -> false)",null,List(2636702334976547),0316-095601-y7ipfe71,null,WriteSerializable,false,"Map(numFiles -> 9, numOutputRows -> 68882, numOutputBytes -> 675647)",null,Databricks-Runtime/

In [0]:
%sql
select count(*) from retaildb.orderdeltatable version as of 0

count(1)
68882


In [0]:
%sql
select count(*) from retaildb.orderdeltatable version as of 8

count(1)
206615


In [0]:
%sql
restore table retaildb.orderdeltatable to version as of 0

table_size_after_restore,num_of_files_after_restore,num_removed_files,num_restored_files,removed_files_size,restored_files_size
675647,9,21,9,1593368,675647


In [0]:
%sql
select count(*) from retaildb.orderdeltatable

count(1)
68882
